In [1]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification
from datasets import load_dataset
import torch
import os
import pandas as pd 
import numpy as np 

import torch.nn as nn
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore")


C:\Users\zhuld\anaconda3\envs\pytorch_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
path = 'C:\\Users\\zhuld\\Dropbox\\EAV'
print(os.listdir(path))

['.ipynb_checkpoints', 'desktop.ini', 'GitHub_Codes', 'meta_data.csv', 'questionnaire.xlsx', 'subject1', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject15', 'subject16', 'subject17', 'subject18', 'subject19', 'subject2', 'subject20', 'subject21', 'subject22', 'subject23', 'subject24', 'subject25', 'subject26', 'subject27', 'subject28', 'subject29', 'subject3', 'subject30', 'subject31', 'subject32', 'subject33', 'subject34', 'subject35', 'subject36', 'subject37', 'subject38', 'subject39', 'subject4', 'subject40', 'subject41', 'subject42', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9', 'temp_subjects.csv']


In [4]:
cnt = 0 
for sub in os.listdir(path):
    if sub[:7] == 'subject':
        for i in os.listdir(path+'/'+sub+'/Audio'):
            print(i)
            cnt += 1 

print(cnt)

002_Trial_02_Speaking_Neutral_Aud.wav
004_Trial_02_Speaking_Anger_aud.wav
006_Trial_02_Speaking_Anger_aud.wav
008_Trial_02_Speaking_Anger_aud.wav
010_Trial_02_Speaking_Anger_aud.wav
012_Trial_02_Speaking_Neutral_aud.wav
014_Trial_02_Speaking_Anger_aud.wav
016_Trial_02_Speaking_Anger_aud.wav
018_Trial_02_Speaking_Anger_aud.wav
020_Trial_02_Speaking_Anger_aud.wav
022_Trial_02_Speaking_Neutral_aud.wav
024_Trial_02_Speaking_Calmness_aud.wav
026_Trial_02_Speaking_Calmness_aud.wav
028_Trial_02_Speaking_Calmness_aud.wav
030_Trial_02_Speaking_Calmness_aud.wav
032_Trial_02_Speaking_Neutral_aud.wav
034_Trial_02_Speaking_Calmness_aud.wav
036_Trial_02_Speaking_Calmness_aud.wav
038_Trial_02_Speaking_Calmness_aud.wav
040_Trial_02_Speaking_Calmness_aud.wav
042_Trial_04_Speaking_Neutral_aud.wav
044_Trial_04_Speaking_Sadness_aud.wav
046_Trial_04_Speaking_Sadness_aud.wav
048_Trial_04_Speaking_Sadness_aud.wav
050_Trial_04_Speaking_Sadness_aud.wav
052_Trial_04_Speaking_Neutral_aud.wav
054_Trial_04_Speakin

In [5]:
file_emotion = []
file_path = []
subjects = []


for sub in os.listdir(path):
    if sub[:7] == 'subject':
        for i in os.listdir(path+'/'+sub+'/Audio'):
            emotion = i.split('_')[4]
            file_emotion.append(emotion)
            file_path.append(path+'/'+sub+'/Audio/'+i)
            subjects.append(sub)

emotion_df = pd.DataFrame(file_emotion, columns=['Emotion'])
path_df = pd.DataFrame(file_path, columns=['Path'])
subjects_df = pd.DataFrame(subjects, columns=['Subject']) 
df = pd.concat([subjects_df, emotion_df, path_df], axis=1)
df

,Subject,Emotion,Path
0,subject1,Neutral,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/002_...
1,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/004_...
2,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/006_...
3,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/008_...
4,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/010_...
...,...,...,...
4195,subject9,Neutral,C:\Users\zhuld\Dropbox\EAV/subject9/Audio/192_...
4196,subject9,Sadness,C:\Users\zhuld\Dropbox\EAV/subject9/Audio/194_...
4197,subject9,Sadness,C:\Users\zhuld\Dropbox\EAV/subject9/Audio/196_...
4198,subject9,Sadness,C:\Users\zhuld\Dropbox\EAV/subject9/Audio/198_...


In [6]:
import re 

def extract_number(s): 
    return int(re.search(r'\d+', s).group())

df['numeric_part'] = df['Subject'].apply(extract_number)
df.sort_values('numeric_part', inplace=True)
df.drop('numeric_part', axis=1, inplace=True)

df

,Subject,Emotion,Path
0,subject1,Neutral,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/002_...
72,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/146_...
71,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/144_...
70,subject1,Neutral,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/142_...
69,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/140_...
...,...,...,...
3628,subject42,Sadness,C:\Users\zhuld\Dropbox\EAV/subject42/Audio/058...
3627,subject42,Sadness,C:\Users\zhuld\Dropbox\EAV/subject42/Audio/056...
3626,subject42,Sadness,C:\Users\zhuld\Dropbox\EAV/subject42/Audio/054...
3636,subject42,Anger,C:\Users\zhuld\Dropbox\EAV/subject42/Audio/074...


In [7]:
from transformers import ASTFeatureExtractor

feature_extractor = ASTFeatureExtractor()

In [8]:
import torchaudio
from torchaudio.transforms import Resample

filepath = df.iloc[0].Path

waveform, sampling_rate = torchaudio.load(filepath)
#waveform = waveform.squeeze().numpy()

print("Original sampling rate is: ", sampling_rate)

target_sampling_rate = 16000
resampler = Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
resampled_waveform = resampler(waveform)

waveform = waveform.squeeze().numpy()
resampled_waveform = resampled_waveform.squeeze().numpy()

print("Original waveform shape:", waveform.shape)
print("Resampled waveform shape: ", resampled_waveform.shape)

Original sampling rate is:  44100
Original waveform shape: (926100,)
Resampled waveform shape:  (336000,)


In [9]:
inputs = feature_extractor(resampled_waveform, sampling_rate=target_sampling_rate, padding="max_length", return_tensors="pt")
input_values = inputs.input_values
print(input_values.shape)

torch.Size([1, 1024, 128])


In [10]:
type(input_values)

torch.Tensor

In [11]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

In [12]:
df.Emotion.unique()

array(['Neutral', 'Anger', 'Sadness', 'Happiness', 'Calmness'],
      dtype=object)

In [12]:
target_sampling_rate = 16000
test_features_list = []
train_features_list = []
test_labels_list = []  # List to store test labels as one-hot encodings
train_labels_list = []  # List to store train labels as one-hot encodings

num_classes = 5

# Mapping of string emotions to integer labels
emotion_to_index = {
    'Happiness': 0,
    'Neutral': 1,
    'Sadness': 2,
    'Anger': 3,
    'Calmness': 4
}

# inverse mapping 
index_to_emotion = {v: k for k, v in emotion_to_index.items()}

for path, sub, em in zip(df.Path, df.Subject, df.Emotion):
    waveform, sampling_rate = torchaudio.load(path)
        
    resampler = Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
    resampled_waveform = resampler(waveform)
        
    waveform = waveform.squeeze().numpy()
    resampled_waveform = resampled_waveform.squeeze().numpy()

    inputs = feature_extractor(resampled_waveform, sampling_rate=target_sampling_rate, padding="max_length", return_tensors="pt")
    input_values = inputs.input_values

    label_idx = emotion_to_index[em]
    #label_one_hot = torch.nn.functional.one_hot(torch.tensor(label_idx), num_classes=num_classes)
    label = emotion_to_index[em]
    
    if sub == 'subject1':
        test_features_list.append(input_values)
        test_labels_list.append(label)
    else: 
        train_features_list.append(input_values)
        train_labels_list.append(label)

In [14]:
test_features = torch.cat(test_features_list, dim=0)
train_features = torch.cat(train_features_list, dim=0)
test_features.shape, train_features.shape

(torch.Size([100, 1024, 128]), torch.Size([4100, 1024, 128]))

In [15]:
test_labels = torch.Tensor(test_labels_list)
train_labels = torch.Tensor(train_labels_list)
test_labels.shape, train_labels.shape

#test_labels = torch.stack(test_labels_list)
#train_labels = torch.stack(train_labels_list)
#test_labels.shape, train_labels.shape

(torch.Size([100]), torch.Size([4100]))

In [16]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_features, train_labels)
test_dataset = TensorDataset(test_features, test_labels)

In [49]:
from torch.utils.data import DataLoader

loader_train = DataLoader(train_dataset, batch_size=16, shuffle=True)
loader_test = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [30]:
for batch_idx, (features, labels) in enumerate(loader_train):
    # Your training code here, using the features and labels
    print(features.shape, labels.shape)
    break

torch.Size([64, 1024, 128]) torch.Size([64])


In [19]:
model 

ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTAttention(
            (attention): ASTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
            (de

In [20]:
model.classifier.dense.in_features

768

In [81]:
model.classifier.dense = torch.nn.Linear(model.classifier.dense.in_features, 5)

# Freeze all the layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classifier layer (assuming 'model.head' is the classifier layer)
for param in model.classifier.dense.parameters():
    param.requires_grad = True

In [82]:
model

ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTAttention(
            (attention): ASTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
            (de

In [21]:
def add_weight_decay(model, weight_decay=1e-5, skip_list=()):
    decay = []
    no_decay = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue  # frozen weights
        if len(param.shape) == 1 or name.endswith(".bias") or name in skip_list:
            no_decay.append(param)
        else:
            decay.append(param)
    return [
        {'params': no_decay, 'weight_decay': 0.},
        {'params': decay, 'weight_decay': weight_decay}]

# weight_decay
skip = {}
if hasattr(model, 'no_weight_decay'):
    skip = model.no_weight_decay()

parameters = add_weight_decay(model, 0.0001, skip)
weight_decay = 0.

optimizer = optim.AdamW(parameters)
loss_fn = nn.CrossEntropyLoss()

In [22]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""

            {value}

    """.format(value=value, max=max))

In [23]:
train_len, test_len = len(train_dataset), len(test_dataset)

def eval_corona(model, loader_val):
  model.eval()

  preds = []
  with torch.no_grad():
    for x, t in loader_val:
      x, t = x.to(device), t.to(device) # comment this line if use option1 dataloader
      logits = model(x).logits
      preds.append(torch.sum(torch.max(logits, dim=1)[1] == t))

  return sum(preds) / test_len

In [24]:
train_len, test_len

(4100, 100)

In [108]:
iters = 10
train_len, test_len = len(train_dataset), len(test_dataset)
pb_train = display(progress(0, iters), display_id=True)
losses = [[]]
accus_train = [[]]
accus_val = []

model = model.to(device)

for i in range(iters):
  for x, t in loader_train:
    x, t = x.to(device), t.long().to(device)
    #x, t = x.to(device), t.to(device).long() # comment this line if use option1 dataloader
    logits = model(x).logits
    #print(logits)
    # print(logits.mean())
    loss = loss_fn(logits, t)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses[-1].append(loss.item())

    with torch.no_grad():
      accus_train[-1].append(torch.sum(torch.max(logits, dim=1)[1] == t))
      #print(torch.max(logits, dim=1)[1])
    # break

  losses[-1] = sum(losses[-1])
  losses.append([])
  accus_train[-1] = sum(accus_train[-1]) / train_len
  accus_train.append([])

  # step LR for next epoch
  #lr_scheduler.step(i + 1)

  accus_val.append(eval_corona(model, loader_test))
  model.train()

  pb_train.update(progress(i, iters))
  print(f'\r[Step {i+1}] loss={loss:.2e} '
        f'\r[Step {i+1}] loss={losses[-2]:.2e} '
        f'train accu={accus_train[-2]:.2%} '
        f'validation accu={accus_val[-1]:.2%} '
        # , end=''
        , flush=True)
  # break

[Step 1] loss=7.30e+01 train accu=51.98% validation accu=48.00% 
[Step 2] loss=6.79e+01 train accu=56.78% validation accu=54.00% 
[Step 3] loss=6.56e+01 train accu=58.02% validation accu=50.00% 
[Step 4] loss=6.27e+01 train accu=60.20% validation accu=55.00% 
[Step 5] loss=6.11e+01 train accu=61.05% validation accu=53.00% 
[Step 6] loss=6.02e+01 train accu=62.63% validation accu=59.00% 
[Step 7] loss=6.00e+01 train accu=61.46% validation accu=52.00% 
[Step 8] loss=5.70e+01 train accu=64.78% validation accu=47.00% 
[Step 9] loss=5.66e+01 train accu=64.73% validation accu=45.00% 
[Step 10] loss=5.56e+01 train accu=65.66% validation accu=53.00% 


In [129]:
save_path = "audioTsub1.pth"
torch.save(model, save_path)

In [51]:
save_path = "audioTsub1.pth"
model = torch.load(save_path)
model = model.to(device)
model

ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTAttention(
            (attention): ASTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
            (de

# Fine tuning 

In [52]:
for param in model.parameters():
    param.requires_grad = True

# Unfreeze the classifier layer (assuming 'model.head' is the classifier layer)
for param in model.classifier.dense.parameters():
    param.requires_grad = True
    
lr = 0.00001 
optimizer = optim.AdamW(parameters)
loss_fn = nn.CrossEntropyLoss()
iters = 10


In [38]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import torch


In [54]:
pb_train = display(progress(0, iters), display_id=True)
losses = [[]]
accus_train = [[]]
accus_val = []
max_val = -1
save_path = "audioTsub1.pth"

for i in range(iters):
  for x, t in loader_train:
    x, t = x.to(device), t.to(device)
    t = t.long()
    #print(x)
    logits = model(x).logits
    # print(logits.mean())
    loss = loss_fn(logits, t)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses[-1].append(loss.item())

    with torch.no_grad():
      accus_train[-1].append(torch.sum(torch.max(logits, dim=1)[1] == t))
      #print(torch.max(logits, dim=1)[1])
    # break

  losses[-1] = sum(losses[-1])
  losses.append([])
  accus_train[-1] = sum(accus_train[-1]) / train_len
  accus_train.append([])

  accus_val.append(eval_corona(model, loader_test))
  cur_val_acc = accus_val[-1]
  if cur_val_acc > max_val:
    max_val = cur_val_acc
    torch.save(model, save_path)

  model.train()

  pb_train.update(progress(i, iters))
  print(f'\r[Step {i+1}] loss={loss:.2e} '
        f'\r[Step {i+1}] loss={losses[-2]:.2e} '
        f'train accu={accus_train[-2]:.2%} '
        f'validation accu={accus_val[-1]:.2%} '
        # , end=''
        , flush=True)
  # break


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [55]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [37]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 4            |        cudaMalloc retries: 4         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  52707 MiB |  52707 MiB | 544771 MiB | 492064 MiB |\n|       from large pool |  52696 MiB |  52696 MiB | 544718 MiB | 492022 MiB |\n|       from small pool |     11 MiB |     12 MiB |     52 MiB |     41 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  52707 MiB |  52707 MiB | 544771 MiB | 492064 MiB |\n|       from large pool |  52696 MiB |  52696 MiB |

In [30]:
with torch.no_grad():
  outputs = model(input_values)

In [36]:
outputs.logits.shape # number of classes 

torch.Size([1, 527])

In [37]:
predicted_class_idx = outputs.logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: Speech


In [41]:
for i in range(1, 528):
    print(model.config.id2label[i])

Male speech, man speaking
Female speech, woman speaking
Child speech, kid speaking
Conversation
Narration, monologue
Babbling
Speech synthesizer
Shout
Bellow
Whoop
Yell
Battle cry
Children shouting
Screaming
Whispering
Laughter
Baby laughter
Giggle
Snicker
Belly laugh
Chuckle, chortle
Crying, sobbing
Baby cry, infant cry
Whimper
Wail, moan
Sigh
Singing
Choir
Yodeling
Chant
Mantra
Male singing
Female singing
Child singing
Synthetic singing
Rapping
Humming
Groan
Grunt
Whistling
Breathing
Wheeze
Snoring
Gasp
Pant
Snort
Cough
Throat clearing
Sneeze
Sniff
Run
Shuffle
Walk, footsteps
Chewing, mastication
Biting
Gargling
Stomach rumble
Burping, eructation
Hiccup
Fart
Hands
Finger snapping
Clapping
Heart sounds, heartbeat
Heart murmur
Cheering
Applause
Chatter
Crowd
Hubbub, speech noise, speech babble
Children playing
Animal
Domestic animals, pets
Dog
Bark
Yip
Howl
Bow-wow
Growling
Whimper (dog)
Cat
Purr
Meow
Hiss
Caterwaul
Livestock, farm animals, working animals
Horse
Clip-clop
Neigh, whinny

KeyError: 527